In [172]:
from urllib.request import urlopen
import string
import re
import numpy as np
import pandas as pd

In [234]:
# settings. Change the semester and year to crawl course info from different term.
SEMESTER = "Fall"
YEAR = "2016"

In [235]:
# We want to crawl the data subject by subject, from A to Z,
# as this is an easy way to create links.

# frist creat a template and concatenate it with A-Z to crawl the links represented by each character (A-Z) on the webpage.
parent = "http://www.columbia.edu/cu/bulletin/uwb/sel/subj-"
sub = list(string.ascii_uppercase)
urls = [parent+item+".html" for item in sub]

# then use these created links to crawl the link of each subject.
suburls = []
for url in urls:
    content = str(urlopen(url).read())   
    suburls+=re.findall(r'href="(/cu/bulletin/uwb/subj/.{4}/_%s%s.html)' % (SEMESTER,YEAR),content)
suburls = ["http://www.columbia.edu"+item for item in suburls]
suburls

[]

In [236]:
# We want to ouput a csv file, so we choose pandas.DataFrame.
# To create a pandas dataframe, we need to first obtain data from
# the webpages and save them into a large dictionary; after that,
# which will be passed to pandas.DataFrame.

# create dictionary
d = {"call":[],"number":[],"name":[],"Mon":[],"Tue":[],"Wed":[],"Thu":[],"Fri":[],"Sat":[],"building":[],"room":[]}

# k: kth url being processed. Will be printed for debugging.
k=-1

# do this for all the urls
for suburl in suburls:
    k=k+1
    content = str(urlopen(suburl).read()) # read the whole html content of the current link
    course = re.findall(r'''%s %s.+?\w{4}<br>(.+?)bgcolor="*?#99''' % (SEMESTER, YEAR), content)[1:] # html data of every course on the webpage are the elements
    
    # paste the course name with section name
    mul = [len(re.findall(r'Section .{2,3}</a>',x)) for x in course] # how many sections each course has
    sec = re.findall(r'''(Section .{2,3})''',content) # section list of each course, to be concatenated with course name
    name = re.findall(r'''%s %s.+?\w{4}<br>(.+?)</''' % (SEMESTER, YEAR),content)[1:] # course name of each course
    res = []
    num = re.findall(r'''<b>%s %s (.+?\w{4})<br>''' % (SEMESTER, YEAR),content) # course number (not call number) of each course
    res2=[]
    for i in range(len(mul)):
        # because some courses have more than one section, the length of course name and the length of call number may not be the same
        # here we multiply each course name and course number with the number of section they have
        # and they will be the final course name list and course number list
        res+=[name[i]]*mul[i]
        res2+=[num[i]]*mul[i]
    d["name"]+=[res[i]+" "+sec[i] for i in range(len(res))] # concatenate with section number
    d["number"]+=res2
    
    d["call"]+=re.findall(r'''<b>Call Number:</b> (.*?\w*?) <b>''',content) # the call number list
    
    # split time info and location info
    time=[]
    location=[]
    for i in range(len(course)):
        timeloca = re.findall(r'''</a><br>\\n <b>Day/Time:</b> (.+?) <b>Location:</b> (.+?)<br>''',course[i]) # list of tuples containing time and corresponding location
        
        # weird cases to handle:
        # some courses have more than one data/time + location -> only use the first line
        # some courses have time but no location (e.g., "To be announced") -> later replace it with ""
        if len(timeloca) == 0:
            time+=[""]*mul[i]
            location+=[" "]*mul[i]
        else:
            if len(timeloca) == mul[i]:
                time+=[x for x,y in timeloca]
                location+=[y for x,y in timeloca]
            else:
                block = re.findall(r'''bgcolor=#DADADA>.+?</td>''',course[i])[1::2]
                for j in range(len(block)):
                    t = re.findall(r'''</a><br>\\n <b>Day/Time:</b> (.+?) <b>Location:</b> .+?<br>''',block[j])
                    l = re.findall(r'''</a><br>\\n <b>Day/Time:</b> .+? <b>Location:</b> (.+?)<br>''',block[j])
                    if len(t) == 0:
                        time+=[""]
                        location+=[" "]
                    else:
                        time+=t
                        location+=l
                    
    time = [x.split(" ") for x in time] # split weekday and time
    
    for i in range(len(time)):
        if "M" in time[i][0]:
            d["Mon"] += [str(time[i][1])]
        else:
            d["Mon"] += [""]
        if "T" in time[i][0]:
            d["Tue"] += [str(time[i][1])]
        else:
            d["Tue"] += [""]
        if "W" in time[i][0]:
            d["Wed"] += [str(time[i][1])]
        else:
            d["Wed"] += [""]
        if "R" in time[i][0]:
            d["Thu"] += [str(time[i][1])]
        else:
            d["Thu"] += [""]
        if "F" in time[i][0]:
            d["Fri"] += [str(time[i][1])]
        else:
            d["Fri"] += [""]
        if "S" in time[i][0]:
            d["Sat"] += [str(time[i][1])]
        else:
            d["Sat"] += [""]
    
    location = [x.replace("ONLINE ONLY"," ") for x in location]
    location = [x.replace("To be announced", " ") for x in location]
    location = [x.split(" ",1) for x in location] # use the first space to split the string, i.e, split room and building
    d["room"]+=[x for x,y in location]
    d["building"]+=[y for x,y in location]
    
    #d["inst"]+=re.findall(r'Instructors*?:</b> (.+?)</td>',content) # this variable has not implemented yet.. the code needs to be modified (it IS wrong as some courses don't have an instructor
    
    # print the debugging info
    for m in list(d.keys()):
        if len(set([len(item) for item in d.values()])) != 1:
            print(str(k)+": "+m+" "+str(len(d[m])))

# pass the dictionary to pd.DataFrame
df = pd.DataFrame(d)
df.tail() # check the last several rows

,Fri,Mon,Sat,Thu,Tue,Wed,building,call,name,number,room


In [230]:
# write the dataframe into csv file
df.to_csv("Accounting.csv",index=False)